In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import datasets

from keras.applications.resnet50 import ResNet50 # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


目錄tree:"D:\3_learn\9_MachineLearning\2_ML100-Days_Data\Final\image_data"  
├── test/  
└── train/    
　　├── daisy/    
　　├── dandelion/    
　　├── rose/    
　　├── sunflower/    
　　└── tulip/

In [2]:
# 設定資料路徑
train_path = "D:/3_learn/9_MachineLearning/2_ML100-Days_Data/Final/image_data/train" 
test_path = "D:/3_learn/9_MachineLearning/2_ML100-Days_Data/Final/image_data/test" 

# 圖形預處理
# image augmentation + 從directory feed資料
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_batches = train_datagen.flow_from_directory(
    train_path, target_size = (32,32), 
    classes = ['daisy','dandelion','rose','sunflower','tulip'], 
    batch_size=10,
    subset = 'training')
print(train_batches.image_shape)

valid_batches = train_datagen.flow_from_directory(
    train_path, target_size = (32,32), 
    classes = ['daisy','dandelion','rose','sunflower','tulip'], 
    batch_size = 10,
    subset = 'validation')
print(valid_batches.image_shape)


Found 2260 images belonging to 5 classes.
(32, 32, 3)
Found 563 images belonging to 5 classes.
(32, 32, 3)


In [3]:
# 影像大小
IMAGE_SIZE = (32, 32)
# 影像類別數，共有 5 個類別
NUM_CLASSES = 5
# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 8
# 凍結網路層數
FREEZE_LAYERS = 2
# Epoch 數
NUM_EPOCHS = 10
# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'

In [6]:
# 以訓練好的 ResNet50 為基礎來建立模型
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top = False, weights = 'imagenet', input_tensor = None,
               input_shape = (IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs = net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True
    
net_final.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(),
              metrics = ['accuracy'])

print(net_final.summary())

STEP_SIZE_TRAIN = train_batches.samples // train_batches.batch_size
STEP_SIZE_VALID = valid_batches.samples // valid_batches.batch_size

history = net_final.fit_generator(generator = train_batches, 
                                  steps_per_epoch = STEP_SIZE_TRAIN, 
                                  validation_data = valid_batches,
                                  validation_steps = STEP_SIZE_VALID, 
                                  epochs = NUM_EPOCHS, verbose = 1)
#score = net_final.evaluate(x_test, y_test, verbose = 0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Epoch 1/10
 17/226 [=>............................] - ETA: 5:00 - loss: 2.7460 - acc: 0.2294

KeyboardInterrupt: 

In [ ]:
#batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
#num_classes = 5 # 類別的數量，Cifar 10 共有 10 個類別
#epochs = 10 # 訓練的 epochs 數量



In [ ]:
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(32,32), classes=['daisy','dandelion','rose','sunflower','tulip'], batch_size=10)

predictions = net_final.predict_generator(test_batches, steps=1, verbose=0)

test_imgs, test_labels = next(test_batches)
test_labels = test_labels[:,0]